In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
from environment import *

# import setting_for_ccle as setting
import setting_for_tcga as setting

path_dict = path(setting)

In [ ]:
selected_signal_matrix = pd.read_table(
    path_dict['selected_signal_matrix_file_path'],
    index_col=0,
)

In [ ]:
k_nmf, k_nmfcc_distance_linkage_ccc, nmfcc__k_x_column = ccal.nmf_consensus_cluster_with_multiple_k(
    selected_signal_matrix,
    setting.NMF_KS,
    n_job=setting.MAX_N_JOB,
    plot_df=False,
    plot_w=selected_signal_matrix.shape[0] < 3200,
    plot_h=selected_signal_matrix.shape[1] < 3200,
    directory_path=path_dict['nmf_directory_path'],
#     n_clustering=1,
)

In [ ]:
w = pd.read_table(
    path_dict['w_file_path'],
    index_col=0,
)

h = pd.read_table(
    path_dict['h_file_path'],
    index_col=0,
)

for wt_or_h, wt_or_h_matrix in (
    (
        'wt',
        w.T,
    ),
    (
        'h',
        h,
    ),
):
    title = 'NMF k={} {}'.format(
        setting.NMF_K,
        wt_or_h,
    )

    if wt_or_h_matrix.shape[1] < 3200:

        ccal.plot_heat_map(
            wt_or_h_matrix,
            normalization_method='-0-',
            normalization_axis=0,
            cluster_axis=1,
            title=title,
            xaxis_title=wt_or_h_matrix.columns.name,
            yaxis_title=wt_or_h_matrix.index.name,
        )
        
    ccal.make_comparison_panel(
        wt_or_h_matrix,
        wt_or_h_matrix,
        title=title,
        name_0=wt_or_h_matrix.index.name,
        name_1=wt_or_h_matrix.index.name,
    )

In [ ]:
for wt_or_h, targets, features in (
    (
        'wt',
        w.T,
        selected_signal_matrix.T,
    ),
    (
        'h',
        h,
        selected_signal_matrix,
    ),
):

    n_target = targets.shape[0]

    ccal.make_match_panels(
        (targets.iloc[i] for i in range(n_target)),
        (
            False,
        ) * n_target,
        (
            False,
        ) * n_target,
        (
            'continuous',
        ) * n_target,
        {'Selected Signal': {
            'df': features,
            'data_type': 'continuous',
            'emphasis': 'high',
        }},
    )